# Preprocessing

In [ ]:
# import library
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.graph_objects as go
import seaborn as sns


Load data

In [ ]:
raw_df = pd.read_csv('data/crawl_data.csv')

# Data overview
Let's provide a brief introduction to the dataset and explain the meaning of each column.

First, we will check the size of data

In [ ]:
raw_df.shape

Hãy xem xét sơ qua một vài dòng đầu của dữ liệu

In [ ]:
raw_df.head(5)

Next, let's delve into the meaning of each column in our DataFrame to gain a better understanding

<h2> Explain the meaning of each column </h2>

- **Unnamed: 0**: the index of original data
- **CARNAME**: name of the car
- **ID**: id of the car
- **Make**: brand that the car is made
- **Model**: model of the car
- **Body color**: body/exterior color of the car
- **Type of finish**: the paint finish or coating applied to the vehicle. 
    - Ex: Metallic finish is a type of paint that contains small metallic particles, typically aluminum flakes, which give the paint a shiny and reflective appearance.
- **Interior color**: color of the car's interior
- **Interior material**: materials used to construct the interior components of the vehicle, typically Leather,Cloth,Alcantara(high-end) 
- **Body**: body style or the overall design and structure of the vehicle (Sedan, SUV, MPV,...)
- **Doors**: number of doors on a vehicle. 4/5 means that a car has 4 main doors (two front doors and two rear doors) and a rear hatch/liftgate
- **Seats**: number of seating positions available for occupants.
- **VIN**: (Vehicle Identification Number) - A unique code assigned to every motor vehicle when it's manufactured, used for identification purposes.
- **Fuel**: the type of fuel the vehicle uses, such as gasoline, diesel, electric, hybrid, etc
- **Transmission**: The type of transmission system the vehicle has, such as automatic, manual, or semi-automatic.
- **Drive type**: Specifies whether the vehicle is front-wheel drive (FWD), rear-wheel drive (RWD), or all-wheel drive (AWD).
- **Power**:  total power output of the vehicle's engine or powertrain, often measured in horsepower (hp) or kilowatts (kW).
- **El. motor power**: The power output of the electric motor in electric or hybrid vehicles.
- **CO2 emissions**: The amount of carbon dioxide emitted by the vehicle, measured in grams per kilometer (g/km).
- **Emission class**: The vehicle's emission standard, indicating its compliance with environmental regulations.
- **Battery type**: The type of battery used in electric or hybrid vehicles, such as lithium-ion.
- **AC charging time**: The time (hour) it takes to charge the vehicle's battery using alternating current (AC).
- **DC charging time**: The time (hour) it takes to charge the vehicle's battery using direct current (DC).
- **Battery warranty (km)**: The distance (in kilometers) covered by the warranty for the vehicle's battery.
- **Range extender**:  A feature in some electric vehicles that includes a backup power source (usually a small internal combustion engine) to extend the vehicle's range.
- **Mileage**: The total distance the vehicle has traveled, often measured in miles or kilometers.
- **First registration**: The date when the vehicle was first registered.
- **Condition**: The overall state or condition of the vehicle, such as new or used.
- **Consumption**: The fuel or energy consumption of the vehicle, often expressed in liters per 100 kilometers or miles per gallon.
- **Price**: The selling price of the vehicle.
- **Currency**: The currency in which the vehicle's price is quoted.
- **Tags**: Special features/function that are equipped on the car.
- **Engine capacity**: The total volume of all the cylinders in the engine (ccm).
- **Valid MOT until**: The date until which the vehicle's Ministry of Transport (MOT) certification is valid.
- **Previous owners**: The number of individuals or entities that have owned the vehicle before the current owner.
- **Engine power**: The power output of the vehicle's electric engine.
- **Battery capacity**: The total energy storage capacity of the vehicle's battery (kWh).
- **Hybrid type**: The specific type of hybrid technology employed by the vehicle, such as parallel hybrid or series hybrid.
- **Electric range**: The distance the vehicle can travel on electric power alone.
- **Warranty until**: The date until which the vehicle is covered by a warranty.
- **Weight**: The total weight of the vehicle, often measured in kilograms or pounds.
- **Country of origin**: The country where the vehicle was manufactured.
- **Secondary drive**: Additional features related to the vehicle's drive system, such as a secondary electric motor in hybrid vehicles.
- **Energy efficiency**:  The efficiency of the vehicle in converting energy into motion.
- **Full service history**: Documentation of all the services and maintenance performed on the vehicle.
- **Battery capacity1**: The total energy storage capacity of the vehicle's battery (Ah) (Ampere-hour).

# Data cleaning

## Check for Duplicates

Let's check for and handle duplicate rows in the dataset.

In [ ]:
#TODO
duplicate_rows = raw_df[raw_df.duplicated()]
# print duplicate rows
print(duplicate_rows)

As we can see, there is no duplicated rows. This is expected because we get data for different links.

## Analyzing Data in Columns
Next, we will analyze and process the data in the columns of the crawled dataset.

- First, we calculate the data type (dtype) of each column in DataFrame `raw_df` and save the result into Series `dtypes` (This Series has the index as the column name).

In [ ]:
#TODO
dtypes = raw_df.dtypes
dtypes

### Remove columns with high missing values and unnecessary columns

- We need to check the data in the columns of the dataset.

In [ ]:
df = raw_df.copy()
df.info()

To enhance clarity, we will now display a list of columns with a significant proportion of missing values

In [ ]:
# Tính tỷ lệ giá trị bị thiếu cho mỗi cột
missing_ratio = df.isnull().mean() * 100

# Lọc ra các cột có tỷ lệ giá trị bị thiếu lớn hơn 60%
missing_column = missing_ratio[missing_ratio > 60]

# In ra tên cột và tỷ lệ giá trị bị thiếu tương ứng
for column_name, missing_percent in missing_column.items():
    print(f"'{column_name}': {missing_percent}%")


Due to the significantly lower reporting of electric vehicles compared to other types of vehicles, there is a substantial amount of missing data for certain attributes exclusive to electric cars.

Primarily, we will address general information columns present in all types of vehicles.

- There are many columns with more than *60%* missing data, such as `Engine power`, `Full service history`, `Hybrid type`, ... Therefore, the columns with a high percentage of missing data will be removed before processing the data.

In [ ]:
missing_column = missing_column.index

#Loại trừ ra 
electric_vehicle_columns = [
    'El. motor power',
    'Battery type',
    'AC charging time',
    'DC charging time',
    'Battery warranty (km)',
    'Battery capacity',
    'Electric range',
    'Battery capacity1'
]

missing_column = missing_column.drop(electric_vehicle_columns )
df = df.drop(missing_column, axis = 1)

- Removing unnecessary columns:
    - `Unnamed:0`: the index number    
    - `Currency`: the currency unit, which is EUR for all entries
    Therefore, we will also remove these columns from the dataset.

In [ ]:
df = df.drop(['Currency','Unnamed: 0'], axis = 1)

Next, we will examine the missing value rates within the primary columns related to electric vehicles. We need to assess how these columns perform in terms of missing values when considering only the subset of electric vehicles. This evaluation will help determine whether these columns can be effectively utilized for analysis.

In [ ]:
# Lọc ra xe điện từ df
df_electric = df[df['Fuel'] == 'Electric']

# Tính toán tỷ lệ phần trăm giá trị bị thiếu cho từng cột
missing_values_total = df[electric_vehicle_columns].isnull().mean() * 100
missing_values_electric = df_electric[electric_vehicle_columns].isnull().mean() * 100

# Tạo biểu đồ
fig = go.Figure()

fig.add_trace(go.Bar(
    x=electric_vehicle_columns,
    y=missing_values_total,
    name='Tổng thể',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=electric_vehicle_columns,
    y=missing_values_electric,
    name='Xe điện',
    marker_color='lightsalmon'
))

# Điều chỉnh giao diện
fig.update_layout(
    title='Phần trăm giá trị bị thiếu theo từng cột',
    xaxis_title='Tên cột',
    yaxis_title='Phần trăm giá trị bị thiếu',
    barmode='group'
)

fig.show()


It can be observed that the missing data in these columns is excessively large for the entire dataset and specifically for the electric vehicle subset. Therefore, the contribution of data from these columns is not substantial enough. Consequently, it is necessary to drop these columns.

In [ ]:
df = df.drop(electric_vehicle_columns, axis = 1)

### Convert Data Type of Columns

- After removing the columns with missing data and unnecessary columns, we need to process the data in the remaining columns. In this section, we will proceed with data processing in these columns.

In [ ]:
df.info()

- There are some columns with invalid data types.
- The columns `Engine capacity`, `Power`, `Battery warranty (km)`, `Mileage`, `Battery capacity`, `CO2 emissions`, `AC charging time`, `DC charging time` and `Seats` should be converted to `int` or `float` data types instead of `string`. Therefore, we will process the data strings to convert them into numerical data types.

In [ ]:
#TODO
column_convert_to_numeric = ['Engine capacity','Power', 'Consumption','Mileage', 'CO2 emissions']
def convert_to_int(column):
    for i in range(len(column)):
        value = column[i]
        if pd.notnull(value):
            column[i] = value.split(' ')[0].strip().replace('\xa0', '')
    column = pd.to_numeric(column, errors='coerce')
    return column
    
# Convert specified columns 
df[column_convert_to_numeric] = df[column_convert_to_numeric].apply(convert_to_int)
# Convert 'Seats' column 
df['Seats'] = pd.to_numeric(df['Seats'], errors='coerce')

In [ ]:
# Rename the columns
new_column_names = {'Engine capacity': 'Engine capacity(ccm)',
                    'Price': 'Price(EUR)',
                    'Power': 'Power(kW)',
                    'Mileage': 'Mileage(km)',
                    'Consumption':'Consumption(l/100km or kWh/100km)',
                    'CO2 emissions': 'CO2 emissions(g/km)'}
df = df.rename(columns=new_column_names)

Process the columns that store month/year data as strings to datetime: 
- First registration
- Warranty until

In [ ]:
def convert_to_datetime(date_str):
    if pd.notnull(date_str):
        date_format = "%m/%Y"
        date_obj = datetime.strptime(date_str, date_format)
        return date_obj
    else:
        return pd.NaT
convert_column = ['First registration']
df[convert_column] = df[convert_column].applymap(convert_to_datetime)

In [ ]:
# After converting data types
df.info()

- Now the data types of the columns are more appropriate. The next step is to examine the distribution of data in the numerical and categorical columns.

### Process numerical columns

- Filter out the columns that contain numerical data. Calculate the missing data rate for each column.

In [ ]:
numeric_df = df.select_dtypes(include=np.number)
missing_percentage = numeric_df.isnull().mean()

print('The percentage of missing values:')
for idx, missing in zip(missing_percentage.index, missing_percentage): 
    print(f'- {idx}: {round(missing * 100, 2)}%')

In [ ]:
# Lọc các cột có giá trị thiếu lớn hơn 0
missing_percentage_filtered = missing_percentage[missing_percentage > 0].sort_values(ascending = False)
# Vẽ bar chart
plt.barh(missing_percentage_filtered.index, missing_percentage_filtered*100)

# Đặt tiêu đề
plt.title('Percentage of Missing Values', fontweight='bold')

# Đặt nhãn trục x
plt.xlabel('Percentage Missing(%)', fontweight='bold')

# Đặt nhãn trục y
plt.ylabel('Column', fontweight='bold')

# Hiển thị biểu đồ
plt.show()

- For the column `Previous owners`, which indicates whether the car has been previously owned or not, the missing values can be replaced with 0, assuming that the missing values correspond to cars that are brand new.
- The missing values in the columnst `Seats` and `Engine capacity`will be replaced with the median value.
- The missing data in the `Consumption` column will be filled with the mode value within each `Fuel` group.

In [ ]:
df['Previous owners'] = df['Previous owners'].fillna(0)

# Xử lí missing values cột Seats
median_seats = df['Seats'].median()
df['Seats'] = df['Seats'].fillna(median_seats)

# Xử lí missing values cột Engine capacity
median_engine_capacity = df['Engine capacity(ccm)'].median()
df['Engine capacity(ccm)'] = df['Engine capacity(ccm)'].fillna(median_engine_capacity)

# Xử lí missing values cột Consumption
df['Consumption(l/100km or kWh/100km)'] = df.groupby('Fuel')['Consumption(l/100km or kWh/100km)'].transform(lambda x: x.fillna(x.mode()[0]))

- Calculate the missing values again and save them into the `missing_percentages` variable.

In [ ]:
numeric_df = df.select_dtypes(include=np.number)
missing_percentage = numeric_df.isnull().mean()

print('The percentage of missing values:')
for idx, missing in zip(missing_percentage.index, missing_percentage): 
    print(f'- {idx}: {round(missing * 100, 2)}%')

- Now let's examine the distribution of data in the numerical columns. We will calculate percentiles with values of 0% (min), 25%, 50%, 75%, and 100% (max) to see how the data is distributed. The results will be saved in the `numeric_col_profile` variable.

In [ ]:
numeric_columns = df.select_dtypes(include=np.number).columns
numeric_col_profile = df[numeric_columns].describe().loc[["min", "25%", "50%", "75%", "max"]]
numeric_col_profile 

- The distribution of the numerical columns seems to be normal, except for the minimum value of the`Engine capacity(ccm)` column which appears to be a bit unusual.

#### Handle outlier

Firstly, let's generate boxplots for all numerical columns to visually inspect and identify any potential outliers

In [ ]:
import math

numeric_columns = df.select_dtypes(include=np.number).columns
num_plots = len(numeric_columns)
num_cols = 3
num_rows = math.ceil(num_plots / num_cols)

fig, axs = plt.subplots(num_rows, num_cols, figsize=(num_cols*8, num_rows*6))

for i, col in enumerate(numeric_columns):
    row = i // num_cols
    col = i % num_cols
    sns.boxplot(x=df[numeric_columns[i]], ax=axs[row, col])
    axs[row, col].set_title('Boxplot of ' + numeric_columns[i])

plt.tight_layout()
plt.show()


We observe that most columns in our dataset exhibit outliers. These outliers are often not data entry errors, as we are dealing with the automotive market where certain vehicles may surpass others in terms of power, price, or fuel efficiency. For instance, electric cars may have lower CO2 emissions compared to their counterparts.

However, within our data, there are instances where the engine power is listed as less than 500 cc. This appears to be a typo, as the automotive market rarely features cars with such low engine capacities. To address this issue, we will handle these erroneous values by removing them from the dataset

But in first, we will output information about the vehicles suspected of having issue

In [ ]:
#Xem thông tin của xe có Engine capacity(ccm) < 500
df[df['Engine capacity(ccm)'] < 500]

After conducting research and verification through Google search, it has been identified that the low engine capacity values are indeed the result of typo errors. Therefore, we will proceed to remove the rows associated with these erroneous entries.

In [ ]:
#xoá các xe có enginer capacity == 1
df = df[df['Engine capacity(ccm)'] >= 800]

### Processing categorical columns

Let's now examine the categorical columns to determine the extent of missing values in each of these columns

In [ ]:
# Lấy ra các cột object xem tỉ lệ misisng value
cate_col = df.select_dtypes(include=['object']).columns

# Tính tỉ lệ mising của mỗi cột
missing_values = (df[cate_col].isnull().mean() * 100)
missing_values = missing_values.sort_values(ascending=True)

fig = go.Figure(data=[go.Bar(
    y=missing_values.index,
    x=missing_values.values,
    orientation='h',
    marker_color='#1f77b4'
)])

fig.update_layout(
    title_text='Phần trăm giá trị bị thiếu của mỗi cột',
    xaxis_title='Phần trăm giá trị bị thiếu',
    yaxis_title='Tên cột',
    yaxis={'autorange': 'reversed'},
    height=800
)

fig.show()


After researching the market, we understand that people categorize the condition of a vehicle based on its mileage. Therefore, we will fill in the missing values for the 'Condition' field based on the 'Mileage' field.
- If `mileage` <= 300, then `Condition` = 'New'
- If `mileage` > 1000, then `Condition` = 'Used'

In [ ]:
df.loc[df['Mileage(km)'] <= 300, 'Condition'] = df.loc[df['Mileage(km)'] <= 300, 'Condition'].fillna('New')
df.loc[df['Mileage(km)'] > 300, 'Condition'] = df.loc[df['Mileage(km)'] > 300, 'Condition'].fillna('Used')

Next, we will fill in the remaining missing values with the mode of each column.

In [ ]:
#Tiếp theo, ta sẽ điền vào các giá trị thiếu còn lại với mode của mỗi cột
fill_with_mode = ['Body color','Doors', 'Tags', 'Emission class','Interior color', 'Interior material', 'Type of finish']

for column in fill_with_mode:
    df[column].fillna(df[column].mode()[0], inplace=True)

After filling in the values, we can observe the distribution of each column.

In [ ]:
#tạo cate_profile
single_opt = df.select_dtypes(include=['object']).columns
single_opt = single_opt.drop(['Tags'])

profiles = {}
for col in single_opt:
    profiles[col] = df[col].value_counts().to_dict()
    
n = len(profiles)
fig, axs = plt.subplots(n//3 + n%3, 3, figsize=(15, 30))

axs = axs.flatten()

for ax, (col, profile) in zip(axs, profiles.items()):
    # Sort the profile dictionary by value and keep only the top 10
    sorted_profile = dict(sorted(profile.items(), key=lambda item: item[1], reverse=True)[:10])
    ax.bar(sorted_profile.keys(), sorted_profile.values(), color='steelblue')
    ax.set_title(f'Distribution of {col}')
    ax.set_ylabel('Count')
    ax.set_xlabel(col)
    ax.tick_params(axis='x', rotation=45)

# Remove unused subplots

fig.delaxes(axs[-1])
fig.delaxes(axs[-2])

plt.tight_layout()
plt.show()


Nhận xét:

Since the majority of the values in the ‘VIN’ column are ‘not published by seller…’, and similarly for the ‘type of finishes’ column, these columns do not provide useful information. Therefore, we will drop them.

In [ ]:
del df['VIN'], df['Type of finish']

Next, we will examine the ‘Tags’ column.

In [ ]:
#Tính phân phố cho các cột multiple selection
def calc_distribution_multiple(column):
    
    # Tách chuỗi bằng dấu phẩy và loại bỏ khoảng trắng thừa và rỗng
    split_series = column.str.split('; ').explode().str.strip()
    split_series = split_series.dropna()

    # Đếm tất cả các giá trị duy nhất
    attribute_distribution = split_series.value_counts()

    # Trả về dictionary với số lượng giá trị khác nhau và phân phối
    return attribute_distribution

# Hiển thị DataFrame
multiple_cate_profile = calc_distribution_multiple(df['Tags'])
multiple_cate_profile

In [ ]:
# Lấy 10 giá trị đứng đầu và 10 giá trị đứng cuối
top_10 = multiple_cate_profile.sort_values(ascending=False).head(10)
bottom_10 = multiple_cate_profile.sort_values(ascending=False).tail(10)

# Vẽ biểu đồ cho 10 giá trị đứng đầu
plt.figure(figsize=(10, 5))
top_10.plot(kind='barh', color='steelblue')
plt.title('Top 10 values')
plt.xlabel('Count')
plt.show()

# Vẽ biểu đồ cho 10 giá trị đứng cuối
plt.figure(figsize=(10, 5))
bottom_10.plot(kind='barh', color='steelblue')
plt.title('Bottom 10 values')
plt.xlabel('Count')
plt.show()


Nhận xét: 

In [ ]:
#Tính missing value của tất cả các cột trong dataframe
missing_values = df.isnull().mean() * 100

### Check the Reasonableness of Data
We can check the reasonableness of the data by examining the emission levels, as theoretically, electric vehicles should have a CO2 emission of zero. Alternatively, we can check the maximum mileage of new vehicles, they should not exceed 300 km.

In [ ]:
#Kiểm tra lượng khí thải CO2 của các xe điện
electric_cars = df[df['Fuel'] == 'Electric']
co2_emissions = electric_cars['CO2 emissions(g/km)']
co2_emissions.value_counts()

In [ ]:
#Kiểm tra số km đã chạy tối đa của xe mới
new_cars = df[df['Condition'] == 'New']
new_cars['Mileage(km)'].max()

The results are consistent with our predictions: Electric vehicles do not emit CO2 when running, so the CO2 emissions of electric vehicles are 0; and new vehicles cannot have a mileage greater than 300 km. Therefore, we can conclude that our data is reasonable.

## Save the cleaned data
    
Finally, we will save the cleaned data to a csv file for further analysis.

In [ ]:
df.to_csv('data/cleaned_data.csv', index = False)